In [1]:
import pandas as pd
import numpy as np
import os
import csv

cwd = os.getcwd()


In [6]:
!cp /Users/saulgrimaldo1/Downloads/create_training_data.py code/.
!cat dta/training_data_sarcastic_new.txt | python code/create_training_data.py > dta/training_data_new.txt

In [10]:
!head dta/training_data_new.txt

id|text|in_reply_to_screen_name|in_reply_to_user_id|in_reply_to_status_id|sarcastic
977819817725087744|RT @tihosay: Your investments are also secured with TIHOSAY indefinitely. In case of possible death, all users can choose as many inheritor…|None|None|None|0
977815351814602752|@transform_ind @narendramodi Loved the #FitIndia movement &amp; #Wellness drive, @narendramodi ji. Kudos to the volunteers who have produced a beautiful animation featuring you who are the best brand ambassador of #Yoga . Suggest a caveat in it about Yoga being for wellness &amp; not just an excercise. #MannKiBaat|transform_ind|771427441793503233|977800283894857728|0
977811721355132929|RT @sexchatgirlz: 🔥 Best Free Online Sex Cams ↴🔥 Free Registration, Free Tokkens↴ ⭐️ https://t.co/Eoje70wLEY⭐️ https://t.co/Eoje70wLEY…|None|None|None|0
977780855421665281|IDMC @idmcgospeluk - Let There Be Love https://t.co/EAQLJFntAo #ugnjamz #gospel #radio #music|None|None|None|0
977803761472212992|#Music #Video #Np #Livebroadc

In [13]:
#df_sarcastic = pd.read_csv('merged_data_v1_1.csv', sep = '|')

df = pd.DataFrame()



with open('dta/training_data_new.txt', 'r') as csvfile:
    linereader = csv.reader(csvfile, delimiter = '|')
    for i, row in enumerate(linereader):
        if i == 0:
            header= row
            df = pd.DataFrame(columns = row)
        else:
            if len(header) != len(row):
                continue
            else:
                addition = {i:j for i, j in zip(header,row)}
                df = df.append(addition, ignore_index=True )
                


df.tail()        

,id,text,in_reply_to_screen_name,in_reply_to_user_id,in_reply_to_status_id,sarcastic
8721,977761572520067072,@FLAVOLS @robbysoave History? It was made by ...,FLAVOLS,99869658,977752228936241152,1
8722,977761845766324226,Yaoi desu ne? this development in the near end...,None,None,None,1
8723,977762632173215745,Scrolling through Facebook when suddenly... I ...,None,None,None,1
8724,977762993244024833,I cant wait to start my fully semi-automatic A...,None,None,None,1
8725,977763216800526336,Can someone make me cum on kik? Add me : REMUD...,None,None,None,1


In [14]:
for i in np.unique(df['in_reply_to_status_id']):
    if len(i) != 18:
        print (i)

None


In [15]:
#### true exmaples : 
### false examples :
import re
status_id_pattern = r'^[0-9]{18}$'
response_name_not_pattern = r'^#'


index_storage = []

for j in df[df['in_reply_to_status_id'] != 'None'].index:
    if (((df['in_reply_to_screen_name'][j][0] != '#')) & 
        bool(re.match(status_id_pattern,df['in_reply_to_status_id'][j]))):
        index_storage.append(j)
    else: continue

len(index_storage)

1462

In [16]:
#want the clean rows that do not have retreats

status_id_pattern = r'^[0-9]{18}$'
response_name_not_pattern = r'^#'


index_alt = []

for j in df[df['in_reply_to_status_id'] == 'None'].index:
    if (bool(re.match(status_id_pattern,df['id'][j]))):
        index_alt.append(j)
    else: continue

len(index_alt)


7264

In [17]:
#subset those data sets that will not have as much trouble

sub_df = df.iloc[index_storage]
sub_df['url']= "https://twitter.com/"+sub_df['in_reply_to_screen_name']+"/status/"+sub_df['in_reply_to_status_id']
#max(index_storage)
sub_df.head()

/Users/saulgrimaldo1/anaconda/envs/ipykernel_py3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,id,text,in_reply_to_screen_name,in_reply_to_user_id,in_reply_to_status_id,sarcastic,url
1,977815351814602752,@transform_ind @narendramodi Loved the #FitInd...,transform_ind,771427441793503233,977800283894857728,0,https://twitter.com/transform_ind/status/97780...
9,977804219980959745,@WickedVision Awesome and WICKED #Easter #WIN 🤞,WickedVision,74410972,977186821657579520,0,https://twitter.com/WickedVision/status/977186...
39,977807233571540992,@11AMdeclaration @stevesmith49 dont worry abou...,11AMdeclaration,755083473984036864,977643870145179648,0,https://twitter.com/11AMdeclaration/status/977...
80,977815716702441472,@mrkiplingcakes My son made me a beautiful ann...,mrkiplingcakes,121185207,977155702518177793,0,https://twitter.com/mrkiplingcakes/status/9771...
121,977810438590152707,@SappheireBot Dont use other people as an excu...,SappheireBot,191502033,977808501559865344,0,https://twitter.com/SappheireBot/status/977808...


In [132]:
#want the clean rows that do not have retreats
!conda install beautifulsoup


Fetching package metadata ...........

PackageNotFoundError: Packages missing in current channels:
            
  - beautifulsoup

We have searched for the packages in the following channels:
            
  - https://repo.continuum.io/pkgs/main/osx-64
  - https://repo.continuum.io/pkgs/main/noarch
  - https://repo.continuum.io/pkgs/free/osx-64
  - https://repo.continuum.io/pkgs/free/noarch
  - https://repo.continuum.io/pkgs/r/osx-64
  - https://repo.continuum.io/pkgs/r/noarch
  - https://repo.continuum.io/pkgs/pro/osx-64
  - https://repo.continuum.io/pkgs/pro/noarch
            



In [18]:
sub_df

,id,text,in_reply_to_screen_name,in_reply_to_user_id,in_reply_to_status_id,sarcastic,url
1,977815351814602752,@transform_ind @narendramodi Loved the #FitInd...,transform_ind,771427441793503233,977800283894857728,0,https://twitter.com/transform_ind/status/97780...
9,977804219980959745,@WickedVision Awesome and WICKED #Easter #WIN 🤞,WickedVision,74410972,977186821657579520,0,https://twitter.com/WickedVision/status/977186...
39,977807233571540992,@11AMdeclaration @stevesmith49 dont worry abou...,11AMdeclaration,755083473984036864,977643870145179648,0,https://twitter.com/11AMdeclaration/status/977...
80,977815716702441472,@mrkiplingcakes My son made me a beautiful ann...,mrkiplingcakes,121185207,977155702518177793,0,https://twitter.com/mrkiplingcakes/status/9771...
121,977810438590152707,@SappheireBot Dont use other people as an excu...,SappheireBot,191502033,977808501559865344,0,https://twitter.com/SappheireBot/status/977808...
197,977783956840972288,@Votebase @NRA https://t.co/fxlFDEum2wGraphic ...,Votebase,944555091520081920,977685642946609152,0,https://twitter.com/Votebase/status/9776856429...
227,977804211390881792,RT @ButterkistUK: To celebrate #InternationalD...,ButterkistUK,243562815,976035806354780160,0,https://twitter.com/ButterkistUK/status/976035...
389,977803074420043776,@DrLee4America #MAGAideaAll of this #NRA name ...,DrLee4America,200163987,977608716697575424,0,https://twitter.com/DrLee4America/status/97760...
582,977783559439052800,@MarlonBrandan https://t.co/fxlFDEum2wGraphic ...,MarlonBrandan,252802936,977642202217222144,0,https://twitter.com/MarlonBrandan/status/97764...
604,977820097912832001,@Michael2014abc @Kahil @DanDuke2013 @WestsideF...,Michael2014abc,2411474370,977819365360926720,0,https://twitter.com/Michael2014abc/status/9778...


In [19]:
from bs4 import BeautifulSoup
import requests
import re
import json
def preceding_tweet(URL_):

    ### TODO  URL_ will be a function of the table 
    #URL_ = "https://twitter.com/verobchfinfan/status/976155584448868353"
    page = requests.get(URL_)

    soup = BeautifulSoup(page.content, 'html.parser')

    html = list(soup.children)[2]

    body = list(html.children)[3]


    ###  entity in tweet that gives a full tweet with low confidence in structure

    p1 = list(body.children)[61]

    p1a = list(p1.children)[5]
    p1b=list(list(p1a.children)[3].children)[1]
    set_of_parent = set(list(p1b.children)[1].get_text().split('\n'))
    set_of_parent = list(map(lambda x: x.strip(),set_of_parent))

   
    ###  entity in tweet that gives a partial tweet with high confidence in structure
    p2 = list(body.children)[65]

    p2a= json.loads(p2.__dict__['attrs']['value'])

    p2a['initialState']['title']

    pattern_1 = r'^[A-Za-z0-9 ]* on Twitter: "'
    pattern_2 = '\u2026'

    if re.match(pattern_1, p2a['initialState']['title']):
        start_index = len(re.match(pattern_1, p2a['initialState']['title']).group(0))
    else:
        start_index = None
    
    
    if pattern_2 in p2a['initialState']['title']:
        stop_index = p2a['initialState']['title'].rfind(pattern_2)
    else: 
        stop_index = None

    partial_tweet = p2a['initialState']['title'][start_index:stop_index]
    
    
    #### pull out tweet preceding

    potential_tweet_list = [el for el in set_of_parent if partial_tweet in el]

    len_counter = 0

    for i in potential_tweet_list:
        current_len = len(i)
        if current_len> len_counter:
            preceding_tweet_x = i
        else:
            pass
    if len(potential_tweet_list)==0:
        return partial_tweet
    else:
        return preceding_tweet_x

In [20]:
storage_pretweet = []

for x in sub_df['url']:
    url = x.strip()
    try:
        storage_pretweet.append(preceding_tweet(url))
    except:
        storage_pretweet.append(None)
    

In [21]:
storage_pretweet

['#TransformingIndia on Twitter: "An animated video created by a group of volunteers of PM @narendramodi performing Yogasana. In his #MannKiBaat , he urged all to join the movement of wellness & fitness with #FitIndia.',
 'Get ready to crash, bash and smash this #Easter - RT & FOLLOW for the chance to #WIN a pair of Wicked Body Bubble Balls. A truly WICKED prize! #competition T&Cs here: https://t.co/HpyxWVvzmr',
 'Nasser 4 Australia (loan only) on Twitter: ""The team\'s integrity and my integrity have come into question."\n\nNo Steve, your premeditated attempts to break the rules, and your [joint] decision to scapegoat the team\'s most junior member by employing him as your proxy, shows you have no integrity. #SAvAUS"',
 "To celebrate the launch of our new TV ad, we're giving away a hamper of Mr Kipling goodies! Tell us what little thing has happened in your life recently that meant the most to win [UK 18+ only. Closes on 27th March at 12pm. Full T&Cs apply: https://t.co/f4QCf60vKg].",

In [22]:
sub_df['first_preceding_tweet'] = storage_pretweet

sub_df.head()

/Users/saulgrimaldo1/anaconda/envs/ipykernel_py3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,id,text,in_reply_to_screen_name,in_reply_to_user_id,in_reply_to_status_id,sarcastic,url,first_preceding_tweet
1,977815351814602752,@transform_ind @narendramodi Loved the #FitInd...,transform_ind,771427441793503233,977800283894857728,0,https://twitter.com/transform_ind/status/97780...,"#TransformingIndia on Twitter: ""An animated vi..."
9,977804219980959745,@WickedVision Awesome and WICKED #Easter #WIN 🤞,WickedVision,74410972,977186821657579520,0,https://twitter.com/WickedVision/status/977186...,"Get ready to crash, bash and smash this #Easte..."
39,977807233571540992,@11AMdeclaration @stevesmith49 dont worry abou...,11AMdeclaration,755083473984036864,977643870145179648,0,https://twitter.com/11AMdeclaration/status/977...,"Nasser 4 Australia (loan only) on Twitter: """"T..."
80,977815716702441472,@mrkiplingcakes My son made me a beautiful ann...,mrkiplingcakes,121185207,977155702518177793,0,https://twitter.com/mrkiplingcakes/status/9771...,"To celebrate the launch of our new TV ad, we'r..."
121,977810438590152707,@SappheireBot Dont use other people as an excu...,SappheireBot,191502033,977808501559865344,0,https://twitter.com/SappheireBot/status/977808...,"I KNOW, RIGHT?!"


In [23]:
index_alt

[0,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,


In [24]:
merged_data = pd.concat([df.iloc[index_alt],sub_df])

print(merged_data.shape)

merged_data.tail(10)

(8726, 8)


,first_preceding_tweet,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,sarcastic,text,url
8695,You are wronghttps://twitter.com/tlorens/statu...,977736963934638080,donttrythis,977698512023375873,15661871,1,@donttrythis Dam there must be a tell of a lot...,https://twitter.com/donttrythis/status/9776985...
8696,Most of us want a ban on ARs and that is it. S...,977739631771963392,TrumpleForskin,977725004203876353,977517676124876800,1,@marcoPalba @CNNPolitics @CNN Yup because they...,https://twitter.com/TrumpleForskin/status/9777...
8698,"Dan the Beep Man 👮‍♂️🚔 on Twitter: ""Don't thin...",977742801655009280,DanTheBeepMan,977742459072647171,960687787300319232,1,@DanTheBeepMan @danabrams Shouldnt have beeped...,https://twitter.com/DanTheBeepMan/status/97774...
8703,Traders need to adapt to the market. Not expec...,977745396167598080,prrobbins,977744244562685953,18427624,1,@prrobbins My wife says the same thing to me.W...,https://twitter.com/prrobbins/status/977744244...
8706,Happy wife. Happy life.,977747505675894784,prrobbins,977746399931568128,18427624,1,@prrobbins Now if my wife &amp; girlfriend cou...,https://twitter.com/prrobbins/status/977746399...
8707,Trevon Bosley is a Chicago teenager who lost h...,977748330485899264,csmonitor,977747635745574913,14857525,1,@csmonitor I call for Hollywood to stop making...,https://twitter.com/csmonitor/status/977747635...
8710,Updated standings. Devils and Panthers win Sat...,977752488639156224,BroadStBull,977745790885072896,128313714,1,@BroadStBull Im sure Hakstol will have the bes...,https://twitter.com/BroadStBull/status/9777457...
8715,None,977759887273922561,JennSmith64,977759126322233347,863505234236022784,1,@JennSmith64 We all LOVE morgane. ( ) https:/...,https://twitter.com/JennSmith64/status/9777591...
8719,Question: When Democrats say voter I.D. is rac...,977761013327163392,Tokaise,977564363207991296,708344070263812096,1,"@Tokaise Well, you know,... ummm, uuuuuuhhh, n...",https://twitter.com/Tokaise/status/97756436320...
8721,They don’t teach history anymore apparently,977761572520067072,FLAVOLS,977752228936241152,99869658,1,@FLAVOLS @robbysoave History? It was made by ...,https://twitter.com/FLAVOLS/status/97775222893...


In [25]:
import re
def clean_text(text):
    text = re.sub(r"(\||\'|\"|\n|#sarcasm|#sarcastic)"," ", str(text),flags=re.IGNORECASE)
    return text

In [26]:
merged_data['first_preceding_tweet_clean'] = merged_data['first_preceding_tweet'].apply(clean_text)


merged_data_clean = merged_data[['text', 'first_preceding_tweet_clean','sarcastic']]

merged_data_clean.to_csv("merged_data_v3.csv", sep = '|', index = False)

In [150]:
merged_data_clean.head(10)

,text,first_preceding_tweet_clean,sarcastic
0,RT @FlamingoCandles: #FreebieFriday RT &amp; f...,nan,0
1,1.#aafaque_ehsens_ideamines_sites Explore our ...,nan,0
2,RT @lancechain: LanceChain is the first 0% com...,nan,0
3,RT @GocasePro: 🔥#Giveaway 12 Hours👉 Click http...,nan,0
4,BTC bullish scenario https://t.co/0DhLN4VAB4 ...,nan,0
5,PrimeHash - New #Bitcoin Mining!* 190 GHs on s...,nan,0
6,RT @relationshipsdw: #Blog 12: Why helicopter ...,nan,0
7,"Want to face your fears and be free to live, l...",nan,0
8,#Health just started trending with 22979 tweet...,nan,0
9,#vintage #Growing up in northern Kenya in the ...,nan,0


In [31]:
!wc -l dta/merged_data_v2.csv

    3774 dta/merged_data_v2.csv
